<a href="https://colab.research.google.com/github/grazh/cart_experiment/blob/main/%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импортируем библиотеки

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from google.colab import drive
from statsmodels.stats.proportion import proportions_ztest

drive.mount('/content/drive')


Считываем данные

In [ ]:
web_data = pd.read_parquet('/content/drive/My Drive/df.parquet')

In [ ]:
def some_info(df):
    print("Some info about dataset\n") 
    
    # Columns
    print("Cloumns:\n{}".format('\n'.join(list(df.columns))))

    # Shape
    print("This dataset has {} features.".format(df.shape[1]))
    print("This dataset has {} observations.\n".format(df.shape[0]))
    
    # Describe
    print("Describe:\n", df.describe(), "\n\n")
    
    # Missing values
    print("Missing values:\n", round(df.isna().sum()/int(df.shape[0]), 8), "\n")
    
    # Data head
    print(df.head())
  
some_info(web_data)

Удалим колонки, которые не содержат значений или не используются

In [ ]:
web_data = web_data.drop(['timestamp', 'payment_type', 'charity_points', 'days_between', 'name', 'filter_source', 'pg_voucher', 'pg_bonus_card', 'pg_cost_original_delivery'], axis=1)
np.unique(web_data['experiment_name'])
web_data = web_data.drop(['experiment_name'], axis=1)

Проверим сетап эксперимента

In [ ]:
print(web_data.groupby('experiment_variant').nunique()['user_pseudo_id'])
test = web_data[(web_data['experiment_variant'].isin(['2', '3'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')]
test2 = web_data[(web_data['experiment_variant'].isin(['2'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')]
test3 = web_data[(web_data['experiment_variant'].isin(['3'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')]
control = web_data[(web_data['experiment_variant'].isin(['0', '1'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')]

for group in [test, test2, test3, control]:
  print(group.nunique()['user_pseudo_id'])

for i in range(4):
  for j in range(i,4):
    print(f'Пересекаются ли группы {i} и {j}: ', bool(set(web_data[web_data['experiment_variant'] == i]['user_pseudo_id']) & set(web_data[web_data['experiment_variant'] == j]['user_pseudo_id'])))


experiment_variant
0     87670
1     87018
2    174784
3    173874
Name: user_pseudo_id, dtype: int64
51002
25675
25327
26409
Пересекаются ли группы 0 и 0:  False
Пересекаются ли группы 0 и 1:  False
Пересекаются ли группы 0 и 2:  False
Пересекаются ли группы 0 и 3:  False
Пересекаются ли группы 1 и 1:  False
Пересекаются ли группы 1 и 2:  False
Пересекаются ли группы 1 и 3:  False
Пересекаются ли группы 2 и 2:  False
Пересекаются ли группы 2 и 3:  False
Пересекаются ли группы 3 и 3:  False


Изучим изменение метрики Средние траты на пользователя

In [ ]:
alpha_ = 0.05

revenue_test = web_data[(web_data['experiment_variant'].isin(['2', '3'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')].groupby('user_pseudo_id').sum(numeric_only=True)['revenue']
revenue_test2 = web_data[(web_data['experiment_variant'].isin(['2'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')].groupby('user_pseudo_id').sum(numeric_only=True)['revenue']
revenue_test3 = web_data[(web_data['experiment_variant'].isin(['3'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')].groupby('user_pseudo_id').sum(numeric_only=True)['revenue']
revenue_control = web_data[(web_data['experiment_variant'].isin(['0', '1'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')].groupby('user_pseudo_id').sum(numeric_only=True)['revenue']


def check_significance(test, control, metric):
    print(f"""Различия между тестом ({round(test.mean(),2)}) и контролем ({round(control.mean(),2)}) в {metric} {['' if stats.ttest_ind(test, control).pvalue < alpha_ else 'не '][0]}являются статзначимыми.
p-value: {round(stats.ttest_ind(test, control).pvalue, 2)}\n""")
  
check_significance(revenue_test, revenue_control, 'средних тратах на пользователя')
check_significance(revenue_test2, revenue_control, 'средних тратах на пользователя')
check_significance(revenue_test3, revenue_control, 'средних тратах на пользователя')
check_significance(revenue_test2, revenue_test3, 'средних тратах на пользователя')


Различия между тестом (2718.33) и контролем (2680.64) в средних тратах на пользователя не являются статзначимыми.
p-value: 0.14

Различия между тестом (2759.43) и контролем (2680.64) в средних тратах на пользователя являются статзначимыми.
p-value: 0.01

Различия между тестом (2676.65) и контролем (2680.64) в средних тратах на пользователя не являются статзначимыми.
p-value: 0.89

Различия между тестом (2759.43) и контролем (2676.65) в средних тратах на пользователя являются статзначимыми.
p-value: 0.01



Изучим изменение метрики Количество заказов на пользователя

In [ ]:
alpha_ = 0.05

purchase_test = web_data[(web_data['experiment_variant'].isin(['2', '3'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')].groupby('user_pseudo_id').nunique()['order_id']
purchase_test2 = web_data[(web_data['experiment_variant'].isin(['2'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')].groupby('user_pseudo_id').nunique()['order_id']
purchase_test3 = web_data[(web_data['experiment_variant'].isin(['3'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')].groupby('user_pseudo_id').nunique()['order_id']
purchase_control = web_data[(web_data['experiment_variant'].isin(['0', '1'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')].groupby('user_pseudo_id').nunique()['order_id']


def check_significance(test, control, metric):
    print(f"""Различия между тестом ({round(test.mean(),2)}) и контролем ({round(control.mean(),2)}) в {metric} {['' if stats.ttest_ind(test, control).pvalue < alpha_ else 'не '][0]}являются статзначимыми.
p-value: {round(stats.ttest_ind(test, control).pvalue, 2)}\n""")
  
check_significance(purchase_test, purchase_control, 'среднем количестве заказов на пользователя')
check_significance(purchase_test2, purchase_control, 'среднем количестве заказов на пользователя')
check_significance(purchase_test3, purchase_control, 'среднем количестве заказов на пользователя')
check_significance(purchase_test2, purchase_test3, 'среднем количестве заказов на пользователя')



Различия между тестом (1.54) и контролем (1.54) в среднем количестве заказов на пользователя не являются статзначимыми.
p-value: 0.43

Различия между тестом (1.56) и контролем (1.54) в среднем количестве заказов на пользователя являются статзначимыми.
p-value: 0.01

Различия между тестом (1.53) и контролем (1.54) в среднем количестве заказов на пользователя не являются статзначимыми.
p-value: 0.25

Различия между тестом (1.56) и контролем (1.53) в среднем количестве заказов на пользователя являются статзначимыми.
p-value: 0.0



Изучим изменение метрики Средняя стоимость заказа

In [ ]:
alpha_ = 0.05

price_test = web_data[(web_data['experiment_variant'].isin(['2', '3'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')].groupby('order_id').sum(numeric_only=True)['revenue']
price_test2 = web_data[(web_data['experiment_variant'].isin(['2'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')].groupby('order_id').sum(numeric_only=True)['revenue']
price_test3 = web_data[(web_data['experiment_variant'].isin(['3'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')].groupby('order_id').sum(numeric_only=True)['revenue']
price_control = web_data[(web_data['experiment_variant'].isin(['0', '1'])) & (web_data['event_name'] == 'purchase') & (web_data['pg_status'] == 'REALIZATION')].groupby('order_id').sum(numeric_only=True)['revenue']


def check_significance(test, control, metric):
    print(f"""Различия между тестом ({round(test.mean(),2)}) и контролем ({round(control.mean(),2)}) в {metric} {['' if stats.ttest_ind(test, control).pvalue < alpha_ else 'не '][0]}являются статзначимыми.
p-value: {round(stats.ttest_ind(test, control).pvalue, 2)}\n""")
  
check_significance(price_test, price_control, 'средней стоимости заказов')
check_significance(price_test2, price_control, 'средней стоимости заказов')
check_significance(price_test3, price_control, 'средней стоимости заказов')
check_significance(price_test2, price_test3, 'средней стоимости заказов')



Различия между тестом (1760.42) и контролем (1743.11) в средней стоимости заказов не являются статзначимыми.
p-value: 0.11

Различия между тестом (1767.85) и контролем (1743.11) в средней стоимости заказов являются статзначимыми.
p-value: 0.05

Различия между тестом (1752.72) и контролем (1743.11) в средней стоимости заказов не являются статзначимыми.
p-value: 0.45

Различия между тестом (1767.85) и контролем (1752.72) в средней стоимости заказов не являются статзначимыми.
p-value: 0.24



Изучим изменение метрики Среднее число ошибок

In [ ]:
alpha_ = 0.05

error_test = web_data[(web_data['experiment_variant'].isin(['2', '3'])) & (web_data['event_name'] == 'purchase_error')].groupby('user_pseudo_id').count()['event_name']
error_test2 = web_data[(web_data['experiment_variant'].isin(['2'])) & (web_data['event_name'] == 'purchase_error')].groupby('user_pseudo_id').count()['event_name']
error_test3 = web_data[(web_data['experiment_variant'].isin(['3'])) & (web_data['event_name'] == 'purchase_error')].groupby('user_pseudo_id').count()['event_name']


def check_significance(test, control, metric):
    print(f"""Различия между тестом ({round(test.mean(),2)}) и контролем ({round(control.mean(),2)}) в {metric} {['' if stats.ttest_ind(test, control).pvalue < alpha_ else 'не '][0]}являются статзначимыми.
p-value: {round(stats.ttest_ind(test, control).pvalue, 2)}\n""")

check_significance(error_test2, error_test3, 'среднем числе ошибок')


Различия между тестом (1.48) и контролем (2.41) в среднем числе ошибок являются статзначимыми.
p-value: 0.0

